## Racetrack with Monte Carlo Off-policy control

In this lab, we will implement the algorithm shown on p.111 of the textbook–Off Policy MC Control, for estimating $\pi^*$. Below is the problem as shown in the textbook:

* Consider driving a race car around a turn like those shown in Figure 5.5. You want to go as fast as possible, but not so fast as to run off the track. In our simplified racetrack, the car is at one of a discrete set of grid positions, the cells in the diagram. The velocity is also discrete, a number of grid cells moved horizontally and vertically per time step. The actions are increments to the velocity components. Each may be changed by +1, -1, or 0 in each step, for a total of nine (3 x 3) actions. Both velocity components are restricted to be nonnegative and less than 5, and they cannot both be zero except at the starting line. Each episode begins in one of the randomly selected start states with both velocity components zero and ends when the car crosses the finish line. The rewards are -1 for each step until the car crosses the finish line. If the car hits the track boundary, it is moved back to a random position on the starting line, both velocity components are reduced to zero, and the episode continues. Before updating the car’s location at each time step, check to see if the projected path of the car intersects the track boundary. If it intersects the finish line, the episode ends; if it intersects anywhere else, the car is considered to have hit the track boundary and is sent back to the starting line. To make the task more challenging, with probability 0.1 at each time step the velocity increments are both zero, independently of the intended increments. Apply a Monte Carlo control method to this task to compute the optimal policy from each starting state. Exhibit several trajectories following the optimal policy (but turn the noise off for these trajectories).

To complete this exercise, we first recognize that there are three parts involved:
1. Setting up the environment
2. Creating the car agent capable of MC control
3. Train the agent and visualize outcome

### Setting Up the Environment

We want to create a grid with a defined area that is our racetrack. We also want to define areas where the start and finish lines are. Anytime when the car agent is out of bound, it is returned to a random position in the start line and the episode continues. 

Below shows a skeleton of a track that is made to model one of the tracks shown in figure 5.5 in the textbook. There are a couple of options in terms of setting up the environment. You may choose to complete the Track class by filling in the functions intentionally left blank, you could toss the template code and build a track environment to your own likings, or you could use the track environment provided by the ```gym``` Python module. 

Note that you will have to install ```gym``` and import it to this lab if you choose to use ```gym```, and you will need to integrate the control algorithm we are building in this lab with the environment.


#### Task 1: Complete the functions in the Track class


The ```Track``` class is equipped with some basic functions such as setting up and visualizing a track. The learning agent can interact with the track to obtain information like: 
   * the state of the cell it is in
        * the states are 'track', 'oob', 'start', 'finish'
   * a random coordinate for a start state
        * every time the agent goes off track, it will need to return to a randomized position on the starting line
    * any other functions you feel would make sense as part of the environment (e.g. if the agent has crossed the finish line or not)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.table import Table

In [ ]:
class Track:
    
  def __init__(self, _h, _w):
    self.w = _w
    self.h = _h
    # set each cell in grid as 'track'
    self.grid = np.array([['track'] * _w for _ in range(_h)], dtype='U8')

  def make_track_1(self):
    """
    sets the track on the left in figure 5.5 in textbook
    """
    # set start line
    self.grid[-1,:] = 'start'
    # set finish line
    self.grid[:,-1] = 'finish'
    # set area to be out of bound
    self.grid[:,0] = 'oob'
    self.grid[0, :] = 'oob'
    self.grid[:5,1] = 'oob'
    self.grid[:4,2] = 'oob'
    self.grid[:2,3] = 'oob'
    self.grid[-18:,1] = 'oob'
    self.grid[-10:,2] = 'oob'
    self.grid[-7:,3] = 'oob'
    self.grid[7:,11:] = 'oob'
    self.grid[8:,10] = 'oob'
    
    # so that row 0 is start line
    self.grid = np.flip(self.grid, axis=0)
    
  def show_grid(self, _show_trajectory=False, _ys=[], _xs=[]):
    """
    visualize the track with cells in  
    colors that coresponse to the cell type;
    also plots a trajectory if provided with one
    ---
    Params:
    _show_trajectory:  bool
                       set to True if there is a trajectory to show
    _ys: a numpy array containing the y values of the trajectory
    _xs: a numpy array containing the x values of the trajectory
    """
    flipped_grid = np.flip(self.grid, axis=0)
    # setting up the colors that goes with cell type
    self.color_dict = {'track': 'white', 
                       'oob': 'silver', 
                       'start': 'red', 
                       'finish': 'green'}
    fig, ax = plt.subplots()
    fig.set_size_inches(6, 10)
    ax.set_xlim(-1/(self.w), self.w+1/(self.w))
    ax.set_ylim(-1/(self.h), self.h+1/(self.h))
    ax.set_axis_off()
    # use the table and cell objects to make grid
    table = Table(ax)
    # table = Table(ax, bbox=[0, 0, 1, 1])
    for (i, j), cell in np.ndenumerate(self.grid):
        table.add_cell(
            i, j, 1/self.w, 1/self.h, fill=True, 
            facecolor=self.color_dict[flipped_grid[i][j]]
        )
    ax.add_table(table)
    if _show_trajectory:
        xs = _xs + 0.5
        ys = _ys + 0.5
        plt.plot(xs, ys, '-')
    plt.show()
    
  def get_cell_state(self, _y, _x):
    """
    returns the state of the cell [_y, _x]
            ('track', 'oob', 'start', 'finish')
    """
    return 0
    
    
  def randomized_start(self):
    """
    this function should return a random point 
    on the starting line
    """
    return 0   


In [ ]:
# Let's take a look at our environment
H = 33
W = 18
track = Track(H, W)
track.make_track_1()
track.show_grid()

### Creating the Car Agent

We want to give the car agent access to neccessary information such as:
* Its position on the grid
* Its velocity
* The availble actions and their constraints 
    * actions are +1, -1, and 0 in each direction on each step
        * this translates to 9 actions: $[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 0], [0, 1], [1, -1], [1, 0], [1, -1]$
        * determine the dimension of the state space and state-action space so that the matrices for Q, C, pi, etc are implemented correctly
    * velocity components must be nonnegative and less than 5, and annot both be zero except at the starting line
        * how would you handle checking the bound?
        * if you had to correct the velocity because of the agent's action, do you record the action that was actually choosen or record the corrected action as if the agent knew about the constraint on its velocity? 

### Implementing the Learning Algorithm

We can also implement off-policy Monte Carlo control algorithm in this next code block. For Monte Carlo, we want to document for each episode, for each time step $t$ the state $S_t$, the action taken $A_t$, and the reward the eagent received after transitioning to the next state $R_{t+1}$. We will only update $Q$ after the agent reaches the terminal state, in our case, the finish line.

Since we want to implement an off-policy version of Monte Carlo, we also want to set up a policy matrix $\pi$ so that we can iterate on it based on the learning from finishing each episode. Note that the agent will not be following $\pi$ during the episode, and will instead follow a behavioral policy $b$, which is set to be any soft-policy in the textbook. Soft-policies can provide _coverage_, a property meaning that any actions taken under $\pi$ will be taken under $b$ with a non-zero probability. 

Below is a template for the ```Agent``` class with a couple of suggested functions. Feel free to complete the functions, or write other functions to complete the neccessary structures for the learning process.


#### Task 2: Complete the functions in the Agent class

Below are some suggested functions if you would prefer more guidance; you can also get rid of the templates and start from scratch. 

In [ ]:
class Agent:
    
    def __init__(self, _env, _eps=0.01, _gamma=0.9):
        
        self.env = _env
        
        # hyperparameters
        self.eps = _eps
        self.gamma = _gamma
        
        # set up the dimension of different data structure for
        # storing state-action values as tuples for easy accessing
        self.q_dim = (???)
        self.s_dim = (???)
        
        # if you don't want to set up the q_dim and s_dim, 
        # please change the line below to work with 
        # how you want to handle the initialization of q and c
        self.q = np.full(self.q_dim, -1000) # arbitrary initialization value
                                              # another arbitrary initialization: np.random.rand()*(1/len(self.actions))
        # record keeping
        self.c = np.zeros(self.q_dim) # used to track cummulative reward
        
        
        # initialize a policy pi
        # note that you will have to complete the generate_policy() function
        self.pi = self.generate_policy() 
        
        # set up storage for neccessary inforamtion such as 
        # agent's position, velocity, set of available actions, and more

    
    def generate_policy(self):
        """
        generate pi based on Q
        """
        return 0
    
    
    def get_argmax(self):
        """
        implement argmax for determine pi and possibly b 
        (depending on what kind of soft-policy b you end up implementing)
        ---
        returns the action with maximum expected value
                
        """
        return 0
        

    def find_possible_actions(self):
        """
        gather actions that will not cause velocity  
        to be more than 4, or negative, or both zeros
        """
        return 0
        
    
    def epsilon_greedy(self):
        """
        implement a soft behavioral policy b 
        using epsilon-greedy
        ---
        returns an action
        """
        return 0
    
    
    def get_next_pos(self):
        """
        accepts an action that is a np array [y, x]
        set the new position with the velocity
        """
        return 0
    
                                
    def get_new_v(self):
        """
        get new velocity from  
        current velocity and the choosen action
        """
        return 0
    
                                
    def mc_step(self):
        """
        handles everything that needs to happen and store
        in a single time step for Monte Carlo control
        """
        return 0
        
    
    def offpolicy_mc_control(self, _episode=1000):
        """
        generates episodes and iteratively updates policy 
        """
        for episode in range(_episode):
            # loop self.mc_step() until terminal
            # update Q and pi
        


## Traning with Monte Carlo

Now that you have set up the agent and the environment, train the agent. Play around with the hyperparameters and the number of episode to see how they impact performance.

You can visualize the agent's trajectory by using the ```show_grid()``` function provided in the ```Track``` class. Set the parameter ```_show_trajectory``` to ```True```, and pass in the y components and the x components of the agent's position history with the parameters ```_ys``` and ```_xs```.

There might be a lot of bugs in your code at first. A good benchmark is that the first episode typically takes about 1000-2000 time steps to complete, and the subsequent episode should take significantly less time steps to complete. 

In [ ]:
# train your agent here